In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from dataclasses import dataclass
import io
import json
import os
from pathlib import Path
from pprint import pprint
import requests
import sys
from typing import Optional

if '..' not in sys.path: sys.path.append('..')

from datasets import load_dataset
import numpy as np
import pandas as pd
from pydantic_yaml import parse_yaml_file_as, to_yaml_file
import torch
from torch import nn
import torch.nn.functional as F
from transformers import BertGenerationEncoder, BertGenerationDecoder, EncoderDecoderModel, BertTokenizer, AutoTokenizer
from transformers.modeling_outputs import Seq2SeqLMOutput, BaseModelOutputWithPastAndCrossAttentions, CausalLMOutputWithCrossAttentions

from mllm.config.model import GenmixBertCfg
from mllm.model.inference import BeamSearch
from mllm.exp.args import GENMIX_BERT_MODEL_CFG_FNAME
from mllm.model.genmix import GenmixBert
from mllm.train.utils import get_squadv2_df, get_squadv2_batch, QnaQuesInp, get_billsum_df

# BERT Double Attention experiments
## Configs and paths

In [5]:
DATA_PATH = Path(os.path.expandvars('$HOME')) / 'data'

bert_model_name = 'bert-base-uncased'
random_seed = 111
inp_len = 128

device_name = 'cpu'
# device_name = 'cuda'

device = torch.device(device_name)
print(device)

cpu


In [8]:
n_seq_enc, n_seq_dec = 1, 1
d_model = 8
enc_emb = torch.randn((n_seq_enc, d_model))
dec_tok_emb = torch.randn((n_seq_dec, d_model))
print(enc_emb)

tensor([[ 1.5893,  1.9158, -2.2604, -0.7318,  0.2784,  1.2774, -1.7792,  1.4911]])


In [10]:
n_heads = 4
d_head = d_model // n_heads
enc_emb_h = enc_emb.reshape((n_seq_enc, n_heads, d_head))
dec_tok_emb_h = dec_tok_emb.reshape((n_seq_dec, n_heads, d_head))
print(enc_emb_h.shape, dec_tok_emb_h.shape)

torch.Size([1, 4, 2]) torch.Size([1, 4, 2])
